# Analyzing and comparing different venues in Budapest and Prague

### Author: Imre Kovacs, imre.kov@gmail.com

## 1. Introduction

Budapest and Prague are two quickly developing cities in East Europe. Their sizes are not too different, Budapest is the 10th largest city in the EU by population, while Prague is the 14th. Both cities are tourist hotspots, welcoming millions of visitors every year. 
They have similarities in their history, too, as both have been the part of the Austro-Hungarian Empire and both has been ruled by communist governments in the second half of the 20th century. <br>
After 1990, Budapest and Prague became fast growing capitals, with more and more business opening, quickly learning how to adjust to the new capitalist era.

### 1.1 Some background information

Budapest is the capital of Hungary, with population of 1.7 million people, living both sides of the Danube. The city has 23 districts, its overall size is 525.2 km2. Budapest's GDP is above Hungarian and EU average, 52,770 USD per capita. <br>
In 2018, the city welcomed around 12 million international visitors. 

Prague is the capital of the Czech Republic, population is 1.3 million people, at the banks for the Vltava river. The city has 22 districts, its size is 496 km2. Prague's GDP is above the national average with 62500 USD per capita. <br>
In 2017, the city welcomed 8.4 million international visitors.

As seen from the data above, both cities are well above their own national and EU average in terms of financial availabilites. Their per capita GDP can be compared e.g. to Berlin, which has a thriving hospitality sector. The same can be said about these two cities, too, both because of their own population and the incoming tourists' needs. 

### 1.2 Problem: Comparing the different hospitality venues of the two cities

As it has been explained before, the two cities has a lot in common, both in their history and actual development. As per Tripadvisor, Budapest has around 1200 restaurants and around 300 cafes. Prague has almost 2000 resturants and 600 coffee places. <br>
I would like to compare the two cities, mainly their districts, based on their hospitality venues (resturants, coffee shops). I would like to find similarities between the cities' districts, but also would like to find major differences, if there are any. <br>
I will visualize the findings, both in lists and maps, trying to make it even more understandable.

### 1.3 Key objectives of analyzing the data set

1, Which are the most similar districts of Budapest and Prague, based on their venues? <br>
2, Which are the biggest differencies between districts of the two cities? <br>
3, Which are the districts with the least or worst rated venues? <br>

## 2. Data Acquisition and sources

### 2.1 Data sources

I will use the following data for this project:

- Districts of Budapest and Prague, with their latitudes and longitudes. <br>
  Source: Wikipedia, I will create two CSV files for them, as they are not available at one page for easy automated scrapping.

- Resturants and coffee shops in each district of the cities <br>
  Source: Foursquare API

### 2.2 Methodology

- I will load the districts geo data from the mentioned CSV files
- Using Foursquare API, I will get all the needed venues for each district, also their ratings
- I will create clusters of the districts based on their hospitality venues and their ratings
- The clusters will be visualized on maps using Folium and in lists, pointing out their similarities and differences